In [58]:
import os 
import pandas as pd
import numpy as np

from LoadData import LoadData
from Filter import Filter

In [59]:
gap_animals = ["S7063", "S7064", "S7069"]
wt_animals = ["S7068", "S7070", "S7071"] 
gap_rec = [f for f in os.listdir() if "GAP.npy" in f]
wt_rec = [f for f in os.listdir() if "WT.npy" in f] 

In [60]:
# Here choose the animal recording and start/end
directory = '/Volumes/Macintosh HD/Users/gokceuzun/Desktop/4. SENE/Honors Project/'
filename = 'S7063_GAP.npy'
start = 15324481
end = 36959040

# Load the recording, slice and filter it
data = LoadData(directory, filename)
unfiltered_data = data.get_dat()[:, start:end + 1]
fltr = Filter(unfiltered_data)
filtered_data = np.array(fltr.butter_bandpass())

In [61]:
print("There are " + str(len(filtered_data[0])) + " many samples.") # Getting the correct number of samples

There are 21634560 many samples.


In [62]:
df = pd.DataFrame(filtered_data).T
df.set_axis(['ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8', 'ch9', 'ch10', 'ch11', 'ch12', 'ch13', 'ch14', 'ch15', 'ch16'], axis="columns", inplace=True)

In [63]:
print("There are " + str(int(len(df)/1252)) + " many epochs.")

There are 17280 many epochs.


In [64]:
# Add epochs and the sample indices to the recording dataframe
epochs = []
for i in range(17280): # int(len(df)/1252)
    for j in range(1252): # no of samples per epoch
        e = "epoch" + str(i + 1)
        epochs.append(e)
df['epochs'] = epochs
df

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16,epochs
0,14.517459,6.050171,9.636889,16.864971,11.159300,8.129794,0.847358,8.844793,13.181217,9.431838,12.489662,12.430479,13.903798,16.700808,5.349366,12.934711,epoch1
1,9.682026,-2.020716,3.714286,9.115627,6.424321,3.375708,0.007191,4.037892,12.174484,6.720867,7.484081,3.319452,6.870806,12.457773,-1.733461,9.000016,epoch1
2,9.376320,-13.766812,-1.509305,5.616394,4.717928,3.787080,0.637890,2.402899,10.294084,3.869864,-2.529693,-10.401293,-7.039740,2.940996,-1.975282,3.784293,epoch1
3,8.812900,-9.693198,-1.186084,8.444033,5.687863,2.525990,-1.989646,3.213470,9.267854,4.136354,-1.352342,-9.558710,-8.962019,2.358410,-3.258398,4.109588,epoch1
4,11.488102,-2.206936,4.440248,17.630630,10.679531,2.937450,-3.099315,8.274583,10.225103,8.609006,10.262874,1.316788,4.709829,7.621674,1.872643,12.689111,epoch1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21634555,-3.646540,-1.778447,-9.393458,-13.525131,-8.299972,1.362023,-0.264822,-7.636986,-2.278264,-2.539186,-17.708328,-28.356850,-21.146596,-11.046030,2188.363504,-3.566296,epoch17280
21634556,-4.962000,-1.549264,-10.945738,-13.697753,-8.539701,1.258169,-1.482945,-9.502496,-1.375945,-2.975565,-21.092770,-31.387665,-24.783426,-12.141887,2262.593972,-5.313137,epoch17280
21634557,-0.042612,-7.248449,-13.445747,-14.554149,-11.216752,4.843350,-0.983400,-6.757396,-3.477490,-1.782986,-17.033068,-26.788262,-18.592876,-12.903829,2194.342554,0.435350,epoch17280
21634558,0.317769,-14.125230,-7.831569,-12.576911,-8.109595,4.569688,0.346244,-2.951969,-9.401787,-5.021864,-12.397588,-18.879573,-12.258143,-9.157854,1595.389344,0.704171,epoch17280


#### An important step will be loading the entire recording and finding a way to store the results of each epoch so that you can easily extract features for particular time indices down the line for clustering and also keeping track of the indices that have been discarded for filtering so that the time indices don't get mixed up. 

In [91]:
# Keeping track of the indices that have been discarded for filtering so that the time indices don't get mixed up!
discarded_indices = fltr.noise_index
discarded_ch1 = discarded_indices[0]
ch = 'ch1'

n = ["no"]*len(df)
df_ch1 = df[[ch, 'epochs']]
df_ch1.insert(2, 'discarded', n)

for i in discarded_ch1:
    df_ch1.at[i, 'discarded'] = "yes"

df_ch1

,ch1,epochs,discarded
0,14.517459,epoch1,no
1,9.682026,epoch1,no
2,9.376320,epoch1,no
3,8.812900,epoch1,no
4,11.488102,epoch1,no
...,...,...,...
21634555,-3.646540,epoch17280,no
21634556,-4.962000,epoch17280,no
21634557,-0.042612,epoch17280,no
21634558,0.317769,epoch17280,no


In [96]:
# Which channels or epochs to use can be selected here: 
d = []
channels = ['ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8', 'ch9', 'ch10', 'ch11', 'ch12', 'ch13', 'ch14', 'ch15', 'ch16']

epochs = ['epoch1', 'epoch2', 'epoch3', 'epoch4', 'epoch5']

for epoch in epochs: # df.epochs
    e = df[df.epochs == epoch]
    #print(epoch)
    a = []
    for channel in channels:
        arr = e[channel].to_numpy()
        a.append(arr)
    d.append(a)
    
df_d = pd.DataFrame(d, columns=channels) 
df_d['epochs'] = epochs
df_d

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16,epochs
0,"[14.517459411141314, 9.68202610913161, 9.37631...","[6.050170734109054, -2.0207160460076317, -13.7...","[9.63688947589803, 3.7142859877897805, -1.5093...","[16.864971058053964, 9.115626692360443, 5.6163...","[11.159300021033165, 6.424321460539171, 4.7179...","[8.129794494053032, 3.375708117759906, 3.78707...","[0.8473580547179342, 0.007191232725786811, 0.6...","[8.844793067805156, 4.037892191399959, 2.40289...","[13.181216879084008, 12.174484120039835, 10.29...","[9.431838352368654, 6.720866792806385, 3.86986...","[12.489661728481714, 7.484080742398889, -2.529...","[12.43047904931879, 3.319452440512472, -10.401...","[13.903798144624062, 6.87080645860974, -7.0397...","[16.700808484146297, 12.457772590411349, 2.940...","[5.349365644732483, -1.7334609100843918, -1.97...","[12.934711059559888, 9.00001607626519, 3.78429...",epoch1
1,"[36.4266790802226, 37.45161835525188, 36.29982...","[17.86926813303952, 10.832831376219836, 7.7981...","[17.373062464403958, 11.208706021879825, 4.759...","[19.368857307744587, 17.61401630303911, 13.627...","[15.657765799993287, 11.904772083306987, 7.934...","[3.931212761200225, 3.169270765225269, 0.96338...","[5.457504634287783, 3.3754144975655223, 0.9960...","[10.199214219449523, 8.591305421801986, 6.7654...","[19.809464391916222, 16.120034123821, 7.988776...","[12.973657905350242, 10.321479527024419, 1.962...","[12.423308377212972, 8.810642081665145, 2.3829...","[16.634932259569403, 10.080922395636092, 2.671...","[5.19427298882477, 4.201516153224029, 0.143936...","[-75.77322003201037, -81.75302162352028, -87.0...","[15.8066342418478, 8.606767405628048, 5.298414...","[6.845480844268209, 6.632760674044208, 1.92068...",epoch2
2,"[12.18718848007577, 12.418511515058585, 11.371...","[-2.9705696272221243, 7.212761223958914, 12.57...","[-8.482443964821652, -4.970348225200457, -2.76...","[-13.628134370476555, -9.327326853881704, -13....","[-0.3623393166640225, 3.017262757508239, 0.454...","[-6.054367122816045, -4.986288905589369, -5.33...","[3.971702121916909, 6.397293644777816, 7.82024...","[-10.992613392955578, -7.5363479576844075, -10...","[13.210989800391015, 17.61315812325442, 24.831...","[-13.447165090579508, -7.643289373531568, -0.6...","[16.19516514939972, 22.80997479624355, 28.9087...","[-5.899296868967301, 2.286931115255762, 7.7584...","[64.62876347871693, 71.1897023801094, 74.17420...","[47.75024909334192, 54.07229956698956, 59.3259...","[-2.355897118388694, 4.408866555097269, 17.776...","[-8.08796715456345, -6.917294175084411, -3.179...",epoch3
3,"[22.45868885159402, 26.375392629176368, 23.832...","[35.19874748429637, 25.15688647414171, 21.2061...","[16.806713043060682, 21.65107429174415, 17.286...","[11.52258381572238, 18.604994211082463, 16.006...","[18.711160134697217, 23.845868189236413, 21.91...","[11.74691086176005, 12.121683137451827, 10.093...","[20.47354979508988, 19.287697222610188, 16.298...","[20.14990211100938, 27.077086601137495, 25.325...","[-5.15292633496397, -1.1498515544579648, -4.26...","[4.014244471835262, 9.808018459871455, 5.50922...","[23.33501353142814, 31.862472654904675, 24.945...","[22.67005213796568, 27.20525553052873, 19.0430...","[12.19538418760882, 20.345319788688478, 12.351...","[32.11537582254555, 42.51703438483007, 36.8274...","[36.35912084543649, 28.34218709352538, 23.0620...","[6.373559548238479, 12.26601981476471, 8.92572...",epoch4
4,"[45.453638891809234, 43.81670438892779, 32.327...","[33.72922942124266, 33.22467792137823, 37.2872...","[80.40987155576158, 78.65233757551383, 70.2879...","[82.18952858916796, 83.19087822451888, 74.0666...","[76.67738813666313, 78.0648691848876, 68.55683...","[27.88888706462962, 25.8918433085489, 18.79166...","[22.830998780588935, 22.897828735725085, 18.84...","[92.30416034657131, 93.45628067489523, 84.8207...","[-9.329944815818173, -10.186481756488067, -12....","[26.087376845997646, 24.061222667330313, 22.15...","[52.29225996318509, 50.75793898648

#### The time domain features will be calculated for each epoch for each channel (and the channels determined by the correlation matrices).  
Time domain features: mean value, median, standard deviation, root mean square (RMS), kurtosis, skewness, variance, max, min etc.